In [42]:
from tensorflow.keras.layers import Activation, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate,Reshape,Add,LSTM,Multiply,Embedding,LSTM, GRU,BatchNormalization

from tensorflow.keras.layers import Bidirectional, LeakyReLU
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam, RMSprop

from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras.callbacks import ModelCheckpoint


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
from gensim.models import Word2Vec
from tensorflow.keras.initializers import Constant
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import text_to_word_sequence,Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences,TimeseriesGenerator

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 
import collections


### Загрузим данные
размер словаря - 84267 слов,количество предложений - 73477, максимальная длинна текста - 398 символов, максимальное количество слов в предложении - 190


In [2]:
text=np.array(pd.read_csv('data.csv'))

### Проведем токенизацию

То есть разобьем исходные предложения на слова

In [3]:
def processText(data):
    tokens=[]
    for line in data:
        newToken=text_to_word_sequence(line[2],filters='!"#$%&amp;()*+,-./:;&lt;=>?@[\\]^_`{|}~\t\n\ufeff',
                                  lower=True,split=' ')
        tokens.append(newToken)
    return tokens

In [4]:
text_tokens=processText(text)
print(text_tokens[0])

['любые', 'разногласия', 'во', 'мнениях', 'скоро', 'улягутся', 'а', 'вы', 'продолжайте', 'делать', 'как', 'делали', 'но', 'постарайтесь', 'не', 'наступать', 'на', 'ноги', 'слишком', 'многим', 'иначе', 'ваши', 'сегодняшние', 'действия', 'сыграют', 'против', 'вас', 'в', 'будущем']


### Заменим слова в предложениях на соответсвующие в словаре индексы

In [5]:
num_words = 20000 #84267
tokenizer = Tokenizer(
    num_words=num_words,
    filters='!"#$%&amp;()*+,-—./:;&lt;=>?@[\\]^_`{|}~\t\n\xa0\ufeff',
    lower=True,
    split=' ',
    char_level=False) 


tokenizer.fit_on_texts(text_tokens) 
sequences = tokenizer.texts_to_sequences(text_tokens)

### Зафиксируем характеристики выборки

In [6]:
max_len=0 # максимальная длина предложения

for seq in sequences:
    l=len(seq)
    if l> max_len:
        max_len=l


vocab_len = len(tokenizer.word_index)
embedding_dim=300 # длина эмбеддинга слова

print('максимальное количество слов в предложении ', max_len)
print('длина словаря ', vocab_len)
print('длина эмбеддинга ',embedding_dim)

максимальное количество слов в предложении  188
длина словаря  84267
длина эмбеддинга  300


### Дополним предожения нулями до одной длины

In [7]:
sequences = pad_sequences(sequences = sequences, maxlen = max_len,padding='post')
print('текст предложения\n',sequences[1][:50])
print('размерность всего массива',sequences.shape)

текст предложения
 [  993     9   459  4145  1640   200     5   440    87     8  1118 12000
     2  1851  8608 14200    38   775     1  2501 13658    14    44 16338
    34     7  1162   452  1178    57     1     7   207    53     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0]
размерность всего массива (73477, 188)


### Считаем сохраненные веса word2vec

In [8]:
w2v = Word2Vec.load("weights/word2vec.model")

### Создадим матрицу эмбеддингов

In [9]:
embedding_matrix = w2v.wv.vectors

scaler=MinMaxScaler((-1,1))
print('до нормализации\n',embedding_matrix[458][:10])

embedding_matrix=scaler.fit_transform(embedding_matrix)
print('после нормализации\n',embedding_matrix[458][:10])

до нормализации
 [-0.6245368  -1.3930552  -3.2870445  -0.53247976 -1.3066322  -4.748493
 -0.9808279  -0.87501806 -4.233118    3.80411   ]
после нормализации
 [ 0.00100999 -0.02932662 -0.27354166  0.06112456 -0.14094234 -0.27567697
  0.05776033 -0.03154409 -0.17836615  0.26318383]


### Создадим обучающую выборку

 Например для текста "The sky was falling due to apocalypse " будет
 
 "The sky" -> was
 
 "sky was" -> falling
 
 " was falling " -> due 
 
 " falling due" -> to
 
 "due to " -> apocalypse
 


In [10]:
def word2idx(word):
    return w2v.wv.vocab[word].index

def idx2word(idx):
    return w2v.wv.index_to_key[idx]


In [13]:
window_step=5
N=500000
step=1
count=0

x=[]
y=[]


for line in sequences:
    if count>=N:
        break
        
    for j in range(0, len(line) - step-window_step, step):
        temp=[]
        for i in range(window_step):
            temp.append(line[j+i])
       
        x.append(temp)
        y.append(embedding_matrix[line[j+window_step]])
    
x=np.array(x)
y=np.array(y)
    
print('x shape ',x.shape)
print('y shape ',y.shape)

x shape  (13372814, 5)
y shape  (13372814, 300)


In [15]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.01,shuffle=False) #, random_state = 42)

In [ ]:
del x_train
del y_train
del x_test
del y_test

## Обучим модель

In [17]:
model= Sequential()

model.add(Embedding(input_dim = vocab_len, 
                    output_dim = embedding_dim, 
                    weights=[embedding_matrix],
                    trainable=False
                             ))
model.add(Dense(300,input_shape=(window_step,embedding_dim)))
model.add(LeakyReLU(0.2))

#modelGRU.add(Embedding(num_words, embedding_size))
#modelGRU.add(SpatialDropout1D(0.2))

model.add(LSTM(128, return_sequences=True))
model.add(LSTM(256,return_sequences=True))
model.add(BatchNormalization())

model.add(LSTM(256,return_sequences=True))
model.add(GRU(512,return_sequences=True))
model.add(BatchNormalization())
        
model.add(LSTM(512,return_sequences=True))
model.add(GRU(1024))
model.add(BatchNormalization())

#modelGRU.add(Bidirectional(LSTM(128)))
#modelGRU.add(LSTM(8,return_sequences=True ))
#model.add(LSTM(128))

model.add(Dense(256))
model.add(LeakyReLU(0.2))
model.add(Dropout(0.2))

model.add(Dense(64))
model.add(LeakyReLU(0.2))

model.add(Dense(embedding_dim,activation = 'sigmoid'))
#model.summary()


In [18]:
optimizer=RMSprop(0.0001)
#optimizer=Adam(0.0001)
model.compile(loss="mse", optimizer=optimizer,metrics=["mae"])
history = model.fit(x, y, batch_size=3000, epochs=50, validation_split=0.1, verbose=1)

Epoch 1/50
4012/4012 [==============================] - 306s 75ms/step - loss: 0.0315 - mae: 0.0998 - val_loss: 0.0113 - val_mae: 0.0569
Epoch 2/50
4012/4012 [==============================] - 301s 75ms/step - loss: 0.0132 - mae: 0.0648 - val_loss: 0.0113 - val_mae: 0.0565
Epoch 3/50
  79/4012 [..............................] - ETA: 4:47 - loss: 0.0132 - mae: 0.0644

KeyboardInterrupt: 

In [94]:
modelGRU = Sequential()
#embedding_layer = w2v.wv.get_keras_embedding(train_embeddings=False)
modelGRU.add(Embedding(input_dim = N, 
                              output_dim = embedding_dim, 
                              
                              input_length =s, 
                             ))
#modelGRU.add(Embedding(num_words, embedding_size))
#modelGRU.add(SpatialDropout1D(0.2))
modelGRU.add(Bidirectional(GRU(128, return_sequences=True)))
modelGRU.add(Bidirectional(GRU(256)))
#modelGRU.add(Bidirectional(LSTM(128)))
#modelGRU.add(LSTM(8,return_sequences=True ))
#modelGRU.add(LSTM(128))
modelGRU.add(Dropout(0.2))

modelGRU.add(Dense(64))
modelGRU.add(LeakyReLU(0.2))

modelGRU.add(Dense(num_words,activation = 'softmax'))
#modelGRU.summary()


NameError: name 'sentStep' is not defined

In [24]:
model.save_weights('modelGRU.h5')

In [19]:
plt.plot(history.history['mse'])
plt.plot(history.history['mae'])

NameError: name 'history' is not defined

In [39]:
words=['Сегодня раки сделали весам хороший']
newWords=np.zeros((1,5))

sents = tokenizer.texts_to_sequences(words)
sents

[[16, 825, 5195, 342, 50]]

In [41]:
n=20

newWords[0]=(sents[0][0],sents[0][1],sents[0][2],sents[0][3],sents[0][4])
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
print(words)
for i in range(n):
    res=model.predict(newWords)
    nextIndex=res.argmax()
    predWord=reverse_word_map[nextIndex]
    print(predWord)
    for j in range(newWords.shape[1]-1):
        newWords[0][j]=newWords[0][j+1]
    newWords[0][2]=nextIndex
    print(nextIndex)
    

['Сегодня раки сделали весам хороший']
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
людьми
60
